In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score, \
    mean_absolute_error, \
    median_absolute_error
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt      
import numpy as np        
import pandas
import matplotlib as plt
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import csv
from datetime import date, timedelta
from collections import defaultdict


In [105]:
Path="Raw_Data/grouped_dates_monterey_avg"
df=pd.read_csv(Path)
monterey_df=pd.DataFrame(df)


In [106]:
monterey_df

,DATE,LATITUDE,LONGITUDE,ELEVATION,EMNT,EMXT,TAVG
0,2000-03,36.574080,-121.786360,64.260000,4.575,26.125000,11.800000
1,2000-04,36.574080,-121.786360,64.260000,5.025,28.475000,13.750000
2,2000-05,36.574080,-121.786360,64.260000,5.125,29.450000,14.625000
3,2000-06,36.569425,-121.769075,71.025000,7.400,29.433333,15.866667
4,2000-07,36.574080,-121.786360,64.260000,8.475,21.825000,15.400000
...,...,...,...,...,...,...,...
236,2019-11,36.637198,-121.759970,57.528571,2.250,28.900000,13.250000
237,2019-12,36.637198,-121.759970,57.528571,1.650,20.850000,12.100000
238,2020-01,36.637198,-121.759970,57.528571,1.950,23.300000,11.200000
239,2020-02,36.664012,-121.766689,44.466667,-1.400,26.650000,12.200000


In [109]:
monterey_df[['Year', 'Month']]=monterey_df['DATE'].str.split('-',1,expand=True)

In [110]:
monterey_yr_mo_avg=monterey_df[['Year','Month','TAVG']]

In [113]:
monterey_yr_mo_avg_pivot=monterey_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [114]:
monterey_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000         NaN        NaN  11.800000  13.750000  14.625000  15.866667   
2001   10.025000  10.675000  12.575000  11.050000  15.450000  15.700000   
2002    8.866667  11.685714  11.433333  12.560000  13.066667  14.542857   
2003   13.571429  11.228571  12.485714  11.916667  14.050000  15.960000   
2004   10.033333  10.785714  14.542857  13.771429  14.428571  15.085714   

                                                                         
Month         07         08         09         10         11         12  
Year                                                                     
2000   15.400000  16.025000  17.400000  14.725000  10.833333  11.900000  
2001   15.925000  15.925000  15.975000  15.700000  13.216667  10.666667  
2002   16.085714  16.428571  17.000000  14.871429  14.514286  11.414286  
2003   16.800000  17.828571  18.342857  16.557143  12.000000  11.428571  
2004   16.700000  17.760000  17.880000  15.140000  12.285714  11.228571

In [115]:
Sacramento=pd.read_csv("Raw_Data/grouped_dates_sacramento_avg.csv")
sacramento_df=pd.DataFrame(Sacramento)
Sandiego=pd.read_csv("Raw_Data/grouped_dates_sandiego_avg.csv")
sandiego_df=Sandiego
Santamaria=pd.read_csv("Raw_Data/grouped_dates_santamaria_avg.csv")
santamaria_df=Santamaria
Santa_rosa=pd.read_csv("Raw_Data/grouped_dates_santarosa_avg.csv")
santarosa_df=Santa_rosa
Stockton=pd.read_csv("Raw_Data/grouped_dates_stockton_avg.csv")
stockton_df=Stockton

In [121]:
sacramento_df[['Year', 'Month']]=sacramento_df['DATE'].str.split('-',1,expand=True)
sacramento_yr_mo_avg=sacramento_df[['Year','Month','TAVG']]
sacramento_yr_mo_avg_pivot=sacramento_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [128]:
sacramento_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000    9.466667  11.000000  13.266667  16.433333  19.433333  23.633333   
2001    7.966667   9.566667  14.433333  13.933333  22.800000  23.500000   
2002    7.333333  10.633333  11.733333  15.100000  18.766667  23.166667   
2003   10.466667  10.233333  13.500000  12.600000  18.766667  23.033333   
2004    8.366667  10.533333  15.800000  17.166667  19.800000  22.966667   

                                                                        
Month         07         08         09         10         11        12  
Year                                                                    
2000   23.000000  23.933333  22.033333  16.666667   9.333333  8.633333  
2001   23.633333  24.200000  22.200000  19.066667  13.466667  9.400000  
2002   25.066667  23.600000  22.966667  17.200000  12.600000  9.900000  
2003   26.766667  24.333333  23.500000  19.400000  10.766667  9.600000  
2004   24.566667  24.666667  22.500000  17.000000  10.933333  8.466667

In [123]:
sandiego_df[['Year', 'Month']]=sandiego_df['DATE'].str.split('-',1,expand=True)
sandiego_yr_mo_avg=sandiego_df[['Year','Month','TAVG']]
sandiego_yr_mo_avg_pivot=sandiego_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [129]:
sandiego_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000   13.658824  13.406667  13.368750  16.481250  18.740000  21.021429   
2001   11.022222  11.255556  14.042105  13.752632  18.505263  20.794444   
2002   11.868182  13.495000  12.829412  14.795000  16.563158  19.244444   
2003   15.664706  13.336842  14.194737  13.673684  16.688889  18.733333   
2004   12.252632  11.678947  16.300000  16.364706  19.029412  19.287500   

                                                                         
Month         07         08         09         10         11         12  
Year                                                                     
2000   21.852941  23.529412  21.922222  17.300000  13.029412  13.964706  
2001   21.476190  22.566667  21.500000  19.131579  15.221053  11.342105  
2002   21.463158  21.566667  22.066667  17.100000  16.605263  11.778947  
2003   23.178947  24.315000  22.300000  20.486667  14.440000  12.300000  
2004   22.680000  22.378947  22.406250  17.443750  13.372222  12.817647

In [124]:
santamaria_df[['Year', 'Month']]=santamaria_df['DATE'].str.split('-',1,expand=True)
santamaria_yr_mo_avg=santamaria_df[['Year','Month','TAVG']]
santamaria_yr_mo_avg_pivot=santamaria_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [130]:
santamaria_yr_mo_avg_pivot.head()

TAVG                                                               \
Month         01    02         03         04         05         06         07   
Year                                                                            
2000   11.400000  11.8  12.100000  13.700000  14.700000  16.900000  16.200000   
2001    9.500000  10.2  12.800000        NaN        NaN        NaN        NaN   
2002    9.450000  12.7  11.500000  13.266667  13.733333  16.100000  18.350000   
2003   14.500000  12.1  13.666667  12.633333  15.500000  16.666667  18.833333   
2004   10.833333  10.9  15.050000  14.500000  16.033333  16.100000  17.800000   

                                                         
Month         08         09    10         11         12  
Year                                                     
2000   16.900000  17.700000  15.0  10.900000  12.100000  
2001         NaN        NaN   NaN  14.150000  10.400000  
2002   16.600000  17.000000  14.2  15.900000  11.466667  
2003   18.833333  18.366667  19.0  13.166667  11.433333  
2004   18.866667  19.433333  15.5  12.350000  13.033333

In [131]:
santarosa_df[['Year', 'Month']]=santarosa_df['DATE'].str.split('-',1,expand=True)
santarosa_yr_mo_avg=santarosa_df[['Year','Month','TAVG']]
santarosa_yr_mo_avg_pivot=santarosa_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [132]:
santarosa_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000    9.671429  10.875000  12.285714  14.542857  17.033333  18.442857   
2001    8.037500   9.333333  13.257143  12.200000  19.350000  20.375000   
2002    8.162500  10.966667  11.285714  13.750000  15.362500  20.100000   
2003   11.344444  10.425000  12.800000  11.177778  16.188889  19.414286   
2004    9.100000   9.962500  15.555556  15.087500  16.950000  19.342857   

                                                                        
Month         07         08         09         10         11        12  
Year                                                                    
2000   18.925000  19.700000  19.757143  15.475000  10.228571  9.575000  
2001   18.900000  20.344444  18.675000  17.875000  12.828571  9.487500  
2002   20.342857  20.285714  19.750000  16.188889  13.288889  9.812500  
2003   21.214286  20.800000  20.500000  18.314286  10.411111  9.162500  
2004   20.287500  21.271429  20.942857  15.633333  11.587500  9.528571

In [126]:
stockton_df[['Year', 'Month']]=stockton_df['DATE'].str.split('-',1,expand=True)
stockton_yr_mo_avg=stockton_df[['Year','Month','TAVG']]
stockton_yr_mo_avg_pivot=stockton_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [133]:
stockton_yr_mo_avg_pivot.head()

TAVG                                                     \
Month         01         02         03     04         05         06   
Year                                                                  
2000    9.866667  11.600000  13.166667  16.10  18.933333  23.166667   
2001    7.200000   9.700000  13.966667  13.60  22.100000  23.433333   
2002    7.033333  10.633333  11.950000  15.15  18.600000  23.000000   
2003   10.100000  10.200000  13.433333  13.40  18.950000  22.733333   
2004    8.133333  10.233333  15.800000  17.00  19.766667  22.766667   

                                                                    
Month         07     08         09         10         11        12  
Year                                                                
2000   22.633333  23.95  22.066667  16.866667   9.566667  8.266667  
2001   23.650000  24.20  22.133333  19.000000  12.966667  8.966667  
2002   24.600000  23.10  23.033333  17.533333  12.233333  9.300000  
2003   25.966667  23.90  23.300000  19.966667  11.100000  9.500000  
2004   24.233333  24.20  22.533333  16.700000  10.633333  8.233333